# Soil Moisture Detection with GNSS-IR

This exercise is adapted from [Soil Moisture in Portales, New Mexico
Station Name: p038](https://github.com/kristinemlarson/gnssrefl/blob/master/notebooks/use-cases/Soil_Moisture/p038-portales_new_mexico.ipynb). The code was updated to reflect changes in the [gnssrefl package](https://pypi.org/project/gnssrefl/).

Getting data from EarthScope's GAGE archive requires authentication. We start with a function to authenticate and download data.

In [ ]:
import os
import requests
from pathlib import Path
 
from earthscope_sdk.auth.device_code_flow import DeviceCodeFlowSimple
from earthscope_sdk.auth.auth_flow import NoTokensError

def get_es_file(url, directory_to_save_file='./', token_path='./'):
    """function to get earthscope data using es-sdk
    modified from https://gitlab.com/earthscope/public/earthscope-sdk

    Parameters
    ----------
    url : string
        url of desired file at gage-data.earthscope.org
    directory_to_save_file : str, optional
        path of directory in which to save the file, by default cwd
    token_path : str, optional
        path of directory in which to save the token, by default './'
    """
    # instantiate the device code flow subclass
    device_flow = DeviceCodeFlowSimple(Path(token_path))
    try:
      # get access token from local path
      device_flow.get_access_token_refresh_if_necessary()
    except NoTokensError:
      # if no token was found locally, do the device code flow
      device_flow.do_flow()
    token = device_flow.access_token

    # request a file and provide the token in the Authorization header
    file_name = Path(url).name

    r = requests.get(url, headers={"authorization": f"Bearer {token}"})
    if r.status_code == requests.codes.ok:
      # save the file
      with open(Path(Path(directory_to_save_file) / file_name), 'wb') as f:
          for data in r:
              f.write(data)
    else:
      #problem occured
      print(f"failure: {r.status_code}, {r.reason}")

## Downloading data

The [EarthScope Data Server](https://data.earthscope.org/archive/) houses the geodetic data archive.

P038 was a Plate Boundary Observatory site with high-precision instrumentation capable of detecting motion to a sub-centimeter level. We will download data from 2017 as a test case.  Note that we are using highrate 1-Hz data.

In [ ]:
# Example URL for requesting data
# https://gage-data.earthscope.org/archive/gnss/highrate/1-Hz/rinex/2017/001/p038/p0380010.17d.Z

for doy in np.arange(1,365):
    #download
    url='https://gage-data.earthscope.org/archive/gnss/highrate/1-Hz/rinex/2017/%03d/p038/p038%03d0.17d.Z' %(doy,doy)
    print('downloading: ', url)
    get_es_file(url, './rinex_data/2017/rinex/p038/')

**Task**: If files are downloaded in parallel, it can reduce the time for the program to complete. Can you create a new function that implements a parallel download? Look at this [solution](https://opensourceoptions.com/use-python-to-download-multiple-files-or-urls-in-parallel/) from OpenSource Options. Add a code cell below with your code.

In [ ]:
# Parallel download

from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool

## add your code

Observation files are Hatanaka compressed and gnssrefl uses the [RNXCMP](https://terras.gsi.go.jp/ja/crx2rnx.html) executable to read these files. To simplify configuration and reduce external dependencies, we can use the [hatanaka](https://pypi.org/project/hatanaka/) package to decompress the files.

In [ ]:
import hatanaka

for doy in np.arange(1,365):
    fn='./rinex_data/2017/rinex/p038/p038%03d0.17d.Z' %(doy)
    hatanaka.decompress_on_disk(fn)

## Environment set up



First, start by importing everything we need to run GNSS-IR:

Set environment

In [ ]:
from gnssrefl.utils import check_environment, set_environment, get_sys

notebook_dir = Path.cwd().parents[1]

# Making sure environment variables are set - this is required to run the gnssrefl code
exists = check_environment()
if exists == False:
    set_environment(
        refl_code=str(notebook_dir),
        orbits=str(notebook_dir / "orbits"),
        exe=str(notebook_dir / "exe"),
    )

# If you are running this locally - make sure the items in the exe folder have execution permissions

# Set local variable of refl_code location
refl_code_loc = os.environ["REFL_CODE"]
print("refl_code location:", refl_code_loc)


%env REFL_CODE=/Users/sophiaparafina/git/spara/gnss-exploration/rinex_data



## GNSS-IR

Start by analyzing the data using the normal reflector height (GNSS-IR) processing and generating SNR files. SNR files contain the satellite number, time, elevation angle, azimuth angle, and SNR data

In [ ]:
from gnssrefl.vwc_cl import vwc
from gnssrefl.quickPhase import quickphase
from gnssrefl.vwc_input import vwc_input
from gnssrefl.daily_avg_cl import daily_avg
from gnssrefl.gnssir_input import make_gnssir_input
from gnssrefl.gnssir_cl import gnssir
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.installexe_cl import installexe
import ipywidgets as widgets
import matplotlib.pyplot as plt

refl_code_loc = os.environ["REFL_CODE"]

station = "p038"
year = 2017
doy = 1
doy_end = 365

rinex2snr(station=station, year=year, doy=doy, doy_end=doy_end, nolook=True);

To understand why L2C data are superior to the L1 data recorded for this receiver, use quickLook. You will need to import it from gnssrefl.

In [ ]:
from gnssrefl.quickLook_cl import quicklook
quicklook(station, year, doy, fr=20)

The analysis parameters are set up with `make_gnssir_input`. We only need the L2C data, so have set the parameter accordingly.

In [ ]:
make_gnssir_input(station, 2017);

Now we run gnssir. This will be needed for estimate a priori reflector heights for the soil moisture code.

In [ ]:
gnssir(station, year, doy, doy_end=doy_end);

## Step 2: Soil Moisture

Then we will use those results to run the soil moisture code.

Please read the soil moisture user manual. It is very short and has a lot of tips that will save you time.

We need a list of satellite tracks to use:

In [ ]:
vwc_input(station, 2017)

This creates a file that is stored in $REFL_CODE/input/p038_phaseRH.txt

Now we estimate the phase for each satellite track on each day:

In [ ]:
quickphase(station, year, doy, doy_end=doy_end)

Finally, convert the phase to volumetric water content.

This will result in 4 plots:

Phase results in geographic coordinates
Daily average phase
Model inputs
Final Results
These will all be saved in $REFL_CODE/Files

In [ ]:
vwc(station, year)